[2024-08-09 Fiddler](https://thefiddler.substack.com/p/can-you-hack-gymnastics)
====================


[Simulations](20240809.go) gives about 96.2% and, for the extra credit, 91.6%.

    $ go run 20240809.go
    9617624/10000000 0.961762
    9167039/10000000 0.916704

Setting things up

In [1]:
%display latex
D_A, X_A, D_B, X_B = var('D_A X_A D_B X_B')
A_s = D_A + X_A
A_p = D_A * X_A
B_s = D_B + X_B
B_p = D_B * X_B

The probability that the relative rankings are the same is

$$ \int_0^{10} \frac{dX_A}{10} \int_0^{10} \frac{dX_B}{10} (H(A_s-B_s)H(A_p-B_p) + H(B_s-A_s)H(B_p-A_p)) $$

but I can't figure out how to express that without getting `TypeError`s.  The integrand is

In [2]:
heaviside(A_s-B_s)*heaviside(A_p-B_p) + heaviside(B_s-A_s)*heaviside(B_p-A_p)

heaviside(D_A*X_A - D_B*X_B)*heaviside(D_A - D_B + X_A - X_B) + heaviside(-D_A*X_A + D_B*X_B)*heaviside(-D_A + D_B - X_A + X_B)

A is ahead of B when adding when $X_B < D_A - D_B + X_A$.

A is ahead of B when multiplying when $X_B < D_AX_A/D_B$.

So the relative rankings are then same when $X_B < \min(D_A-D_B+X_A,D_AX_A/D_B)$ or $X_B > \max(D_A-D_B+X_A,D_AX_A/D_B)$.

In [3]:
solve(D_A-D_B+X_A == D_A*X_A/D_B, X_A)

[X_A == D_B]

Assuming $D_A > D_B$, then when $X_A < D_B$, the relative rankings are the same when
$X_B < D_AX_A/D_B$ or $X_B > D_A-D_B+X_A$, and when $X_A > D_B$, the relative rankings
are them same when $X_B < D_A-D_B+X_A$ or $X_B > D_AX_A/D_B$.

If $X_A < D_B$, the probability that $X_B < D_AX_A/D_B$ is $D_AX_A/D_B\cdot1/10$, and
the probability that $X_B > D_A-D_B+X_A$ is 0 when $D_A-D_B+X_A > 10$, or $X_A > 10 - (D_A-D_B)$,
and $(10 - (D_A-D_B+X_A))/10$ when $X_A < 10-(D_A-D_B)$.

If $X_A > D_B$, the probability that $X_B < D_A-D_B+X_A$ is $\min(1,(D_A-D_B+X_A)/10)$, and
the probability that $X_B > D_AX_A/D_B$ is 0 when $D_AX_A/D_B > 10$, or $X_A > 10D_B/D_A$,
is $\max(0,(10 - D_AX_A/D_B)/10)$.

The probability that the relative rankings are the same is

In [4]:
p(D_A,D_B) = integral(D_A*X_A/D_B*1/10, X_A, 0, D_B)/10 \
    + integral((10 - (D_A-D_B+X_A))/10, X_A, 0, D_B)/10 \
    + integral((D_A-D_B+X_A)/10, X_A, D_B, 10-(D_A-D_B))/10 \
    + integral(1, X_A, 10-(D_A-D_B), 10)/10 \
    + integral((10-D_A*X_A/D_B)/10, X_A, D_B, 10*D_B/D_A)/10
p

(D_A, D_B) |--> -1/200*D_A^2 - 1/100*(D_A - 10)*D_B + 1/200*(D_A - 20)*D_B + 1/200*D_A*D_B + 1/200*D_B^2 + 1/10*D_A - 1/10*D_B + 1/2*D_B/D_A + 1/2

In [5]:
(p(6,5), numerical_approx(p(6,5)))

(577/600, 0.961666666666667)

That agrees reasonably with the results of the simulations.

Extra credit
------------
Integrating over $D_A$ and $D_B$, assuming $D_A > D_B$, then doubling to count the $D_A < D_B$ cases,

In [6]:
extra_credit = 2*integral(integral(p, D_B, 0, D_A)/10, D_A, 0, 10)/10
(extra_credit, numerical_approx(extra_credit))

(11/12, 0.916666666666667)

This also reasonably agrees with the results of the simulations.